In [1]:
!pip install geopy
!pip install tqdm
!pip install geocoder

In [2]:
# Import pandas
import pandas as pd
import geocoder as gc
from tqdm import tqdm
#from geopy.geocoders import Nominatim
#from geopy.distance import vincenty

# Import BoxPlot, output_notebook, and show from bokeh.charts
from bokeh.charts import BoxPlot, Donut, Bar, Histogram, output_notebook, show
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend
from bokeh.layouts import gridplot, row
from bokeh.models import HoverTool
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import ColumnDataSource


In [3]:
# Assign spreadsheet filename: file
file = 'imd_student_blind.xlsx'

# Load spreadsheet: xl
xl = pd.ExcelFile(file)

# Print sheet names
print(xl.sheet_names)

['Sheet1']


In [4]:
# Load a sheet into a DataFrame by index: df
df = xl.parse(0)

# Print the head of the DataFrame df
df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado


In [5]:
df.columns

Index(['a_ID', 'CEP', 'ano_ingresso', 'periodo_ingresso', 'status',
       'ano_disciplina', 'periodo_disciplina', 'nota', 'disciplina_ID',
       'status.disciplina'],
      dtype='object')

In [6]:
df.shape

(4842, 10)

In [7]:
#Cópia por valores, não mexer no DF principal
disciplinas = df.copy()

disciplinas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4842 entries, 0 to 4841
Data columns (total 10 columns):
a_ID                  4842 non-null int64
CEP                   4842 non-null int64
ano_ingresso          4842 non-null int64
periodo_ingresso      4842 non-null int64
status                4842 non-null object
ano_disciplina        4842 non-null int64
periodo_disciplina    4842 non-null int64
nota                  4842 non-null float64
disciplina_ID         4842 non-null int64
status.disciplina     4842 non-null object
dtypes: float64(1), int64(7), object(2)
memory usage: 416.1+ KB


In [8]:
disciplinas.columns.values.tolist()

['a_ID',
 'CEP',
 'ano_ingresso',
 'periodo_ingresso',
 'status',
 'ano_disciplina',
 'periodo_disciplina',
 'nota',
 'disciplina_ID',
 'status.disciplina']

In [9]:
#Retira informações desnecessárias para essa análise

disciplinas.drop(disciplinas.columns[0:4], axis=1, inplace=True)

In [10]:
#Objetivo 1 -> Média da disciplina X por período e ano
#Objetivo 2 -> Taxa de trancamento e cancelamento por período e ano
disciplinas.columns.values.tolist()

['status',
 'ano_disciplina',
 'periodo_disciplina',
 'nota',
 'disciplina_ID',
 'status.disciplina']

In [11]:
disciplinas = disciplinas[disciplinas['status'] == 'ATIVO']
disciplinas.head()

,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
3,ATIVO,2014,2,6.1,0,Aprovado
4,ATIVO,2014,1,3.0,0,Reprovado
5,ATIVO,2014,2,7.2,0,Aprovado
8,ATIVO,2014,2,8.9,0,Aprovado
12,ATIVO,2014,2,5.6,0,Aprovado


In [12]:
disciplinas_categorias =  disciplinas.groupby(['ano_disciplina', 'periodo_disciplina', 'disciplina_ID']).mean()

# convert the index to a column
disciplinas_categorias.reset_index(inplace = True )
disciplinas_categorias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
ano_disciplina        42 non-null int64
periodo_disciplina    42 non-null int64
disciplina_ID         42 non-null int64
nota                  42 non-null float64
dtypes: float64(1), int64(3)
memory usage: 1.4 KB


In [13]:
disciplinas_categorias[disciplinas_categorias['disciplina_ID'] == 0]

,ano_disciplina,periodo_disciplina,disciplina_ID,nota
0,2014,1,0,5.872727
6,2014,2,0,5.625676
13,2015,1,0,6.126316
19,2015,2,0,5.486555
26,2016,1,0,4.139394
33,2016,2,0,5.750758


In [14]:
# Make a box plot: p
p = BoxPlot(disciplinas_categorias, values='nota', label='disciplina_ID', color='disciplina_ID',
             title='Distribuição da média de notas das disciplinas',
             legend='bottom_right')

# Set the y axis label
p.yaxis.axis_label='Média por período'
p.add_tools(HoverTool(tooltips=[("Nota", "$y")]))

# Call the output_notebook() 
output_notebook()
show(p)

Loading BokehJS ...

In [15]:
ps = []
#tbs = []

for atual in range(0,7):
    p = BoxPlot(disciplinas_categorias[disciplinas_categorias['disciplina_ID'] == atual], values='nota', label='periodo_disciplina',
                        color='periodo_disciplina', title='Ddisciplina '+str(atual), legend='bottom_right')
    p.yaxis.axis_label='Média por período'
    p.add_tools(HoverTool(tooltips=[("Nota", "$y")]))
    
    #temp = Panel(child=p, title='Ddisciplina '+str(atual))
    
    #tbs.append(temp)
    ps.append(p)
    
# Make a box plot: p


# Set the y axis label

grid = gridplot([ps[0:2], ps[2:4], ps[4:6], [None, ps[6]]], sizing_mode='scale_width')
#grid = gridplot([[ps[0], ps[1]], [ps[2], ps[3]], [ps[4], ps[5]], [None, ps[6]]], sizing_mode='scale_width')

#tabs = Tabs(tabs=[ tbs[0], tbs[5] ],sizing_mode='scale_width' )

#show(tabs)
# show the results
show(grid)

In [16]:
##Quantidade de alunos ativos/Trancados/cancelados/...
count_series = pd.DataFrame(columns=('Status', 'Count'))
count_series["Status"] = df["status"].unique().tolist()
tam = len(df)

for atual in range(0,len(count_series)):
    count_series.iloc[atual]["Count"] = df[df["status"] == count_series.iloc[atual]["Status"]]["status"].count()

def porcent(val):
    p = (val*100)/tam
    return p
    
graph = Donut(count_series, label="Status", values="Count", title="Situação dos alunos" )
graph.add_tools(HoverTool(tooltips=[("Alunos", str(count_series.iloc[1]["Count"])+"/"+str(tam)), #falta trocar o 1 pelo número de alunos da categoria
                                    ("Porcentagem", str(porcent(1000))) #falta trocar o 1000 pelo numero de alunos da categoria
                                   ]
                         ))

show(graph)

In [17]:
###Verificar desistência
def popula(string):
    aux = []
    for atual in range (2014, 2017):
        aux.append(df[df["ano_ingresso"] == atual][df["status"] == string]["status"].count())
    return aux

count_desist = pd.DataFrame(columns=('ANO', 'CANCELADO', 'ATIVO', 'TRANCADO', 'CONCLUIDO', 'FORMANDO', 'FORMADO'))
count_desist["ANO"] = [2014, 2015, 2016]

for atual in count_desist.columns.values:
    if atual != "ANO":
        count_desist[atual] = popula(atual)

print(count_desist)



    ANO  CANCELADO  ATIVO  TRANCADO  CONCLUIDO  FORMANDO  FORMADO
0  2014        584    843       112         13        38       10
1  2015        329   1530       114          7         0        0
2  2016        134   1067        56          0         5        0


/home/danielmarx/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [18]:

BLEND = blend('CANCELADO', "ATIVO",  "TRANCADO",  "CONCLUIDO",  "FORMANDO",  "FORMADO",
              name='unidade', labels_name='Situacao')

bar = Bar(count_desist, values=BLEND,
          label=cat(columns='ANO', sort=False),
          stack=cat(columns='Situacao', sort=False),
          color="Situacao",
          legend='bottom_center',
          title="Índice de formação e desitencia de alunos por ano",
          ylabel = "Alunos",
          tooltips=[('Tipo', "$y")]) ##Corrigir isso também

output_notebook()

show(bar)

Loading BokehJS ...

In [19]:
###Verificar NOtas ENEM
ps = []
temp = ["nota", "enen-nota", "enem-matematica", "enem-redacao"]

for atual in range(0, len(temp)):
    p = BoxPlot(disciplinas_categorias, values=temp[atual], label='disciplina_ID',
                        color='disciplina_ID', title=str(temp[atual])+" x Desempenho em disciplinas ", legend='bottom_right')
    p.yaxis.axis_label='Média por período'
    p.add_tools(HoverTool(tooltips=[("Nota", "$y{f.fff}")]))
    
    #temp = Panel(child=p, title='Ddisciplina '+str(atual))
    
    #tbs.append(temp)
    ps.append(p)
    

grid = gridplot([ps[0:2], ps[2:4]], sizing_mode='scale_width')

output_notebook()
show(grid)

KeyError: 'enen-nota'

In [20]:
#Verificar distancias
distancias = df[df["status"] == "ATIVO"].copy()
distancias["LAT"], distancias["LON"], distancias["KM"] = [0,0,0]
distancias = distancias.reset_index()
uf = gc.google("59064741").latlng

for i in tqdm(range(0,100)): ## trocar o range por range(len(distancias))
    st = distancias.loc[i,'CEP']
    g = gc.google(st)
    if g.lat == None:
        distancias.loc[i, "LAT"] = 0
    elif g.lng == None:
        distancias.loc[i, "LON"] = 0
    else:
        distancias.loc[i, "LON"] = g.lng
        distancias.loc[i, "LAT"] = g.lat
print("Completo")

distancias.to_csv('LatLong_Alunos.csv', encoding="utf-8")
##Salva tabela criada
    
#UFRN 59064-741
distancias.head()

  5%|▌         | 5/100 [00:05<01:41,  1.06s/it]


KeyboardInterrupt: 

In [59]:
distancias =  pd.read_csv('LatLong_Alunos.csv', encoding="utf-8", index_col=0)
    
for atual in tqdm(range(0,100)): ## trocar o range por range(len(distancias))
    lt = distancias.loc[atual, "LAT"]
    ln = distancias.loc[atual, "LON"]
    if lt != 0. and ln != 0.:
        compare = (lt, ln)
        #print(vincenty(uf, compare).km)
        distancias.loc[atual, "KM"] = vincenty(uf, compare).km
    
distancias.head()

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 1080.22it/s]


,index,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,LAT,LON,KM
0,3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,0,0,0
1,4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,0,0,0
2,5,3,59088150,2014,1,ATIVO,2014,2,7.2,0,Aprovado,0,0,0
3,8,5,59112430,2014,1,ATIVO,2014,2,8.9,0,Aprovado,0,0,0
4,12,7,59020330,2014,1,ATIVO,2014,2,5.6,0,Aprovado,0,0,0


In [60]:
def binSearch(array, ano, periodo, element_to_search):
    for index in range(len(array)):
        if((periodo == array[index]['PERIODO'] and ano == array[index]['ANO']) and element_to_search == array[index]['DISCIPLINA']):
            return index
    return -1

In [61]:
import copy

def catchTRA(array, tra):
    dict_model = {'ANO': '', 'PERIODO': '', 'DISCIPLINA': 0, 'NUMERO_DE_ALUNOS': 0, 'PORCENTAGEM': 0}
    anos=  [2014, 2015, 2016]
    periodos = [1, 2]
    for ano in anos:
        for periodo in periodos:
            temp_df1 = df[df['status.disciplina'] == tra]
            temp_df2 = temp_df1[temp_df1['periodo_disciplina'] == periodo]
            temp_df3 = temp_df2[temp_df2['ano_disciplina'] == ano]
            #print(temp_df2)
            for index in range(len(temp_df3)):
                disciplina = temp_df3.iloc[index]['disciplina_ID']
            
                bin_result = binSearch(array, ano, periodo, disciplina)
                if bin_result == -1:
                    temp_dict = copy.copy(dict_model)
                    temp_dict['ANO'] = ano
                    temp_dict['PERIODO'] = periodo
                    temp_dict['DISCIPLINA'] = disciplina
                    temp_dict['NUMERO_DE_ALUNOS'] = 1
                    array.append(temp_dict)
                else:
                    array[bin_result]['NUMERO_DE_ALUNOS'] += 1
    
    for index in range(len(array)):
        index_total_estudantes = df[df['ano_disciplina'] == array[index]['ANO']]
        index_total_estudantes = index_total_estudantes[index_total_estudantes['periodo_disciplina'] == array[index]['PERIODO']]
        index_total_estudantes = index_total_estudantes[index_total_estudantes['disciplina_ID'] == array[index]['DISCIPLINA']]
        index_total_estudantes = len(index_total_estudantes)
    
        array[index]['PORCENTAGEM'] = (array[index]['NUMERO_DE_ALUNOS']/index_total_estudantes) * 100

In [62]:
reprovados = []

catchTRA(reprovados, 'Reprovado')

In [63]:
aprovados = []

catchTRA(aprovados, 'Aprovado')

### Graficos de aprovados e reprovados

In [64]:
import numpy as np
import scipy.special

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file

def cria_graficos_barras_apro_repro(array, disciplina, titulo_grafico):
    dados = []
    
    anos=  [2014, 2015, 2016]
    periodos = [1, 2]
    
    for ano in anos:
        for periodo in periodos:
            for index in range(len(array)):
                if (array[index]['ANO'] == ano and array[index]['PERIODO'] == periodo) and array[index]['DISCIPLINA'] == disciplina:  
                    dados.append(array[index]['PORCENTAGEM'])   

    data = {
    'semestres': ['2014.1', '2014.2', '2015.1', '2015.2', '2016.1', '2016.2'],
    'porcentage': dados
}

    # table-like data results in reconfiguration of the chart with no data manipulation
    bar2 = Bar(data, values='porcentage', label=['semestres'], title=titulo_grafico, plot_width=400)

    output_file("stacked_bar.html")
    show(row(bar2))

In [65]:
cria_graficos_barras_apro_repro(aprovados, 0, 'Aprovados disciplina 0')
cria_graficos_barras_apro_repro(aprovados, 1, 'Aprovados disciplina 1')
cria_graficos_barras_apro_repro(aprovados, 2, 'Aprovados disciplina 2')
cria_graficos_barras_apro_repro(aprovados, 3, 'Aprovados disciplina 3')
cria_graficos_barras_apro_repro(aprovados, 5, 'Aprovados disciplina 5')
cria_graficos_barras_apro_repro(aprovados, 6, 'Aprovados disciplina 6')

INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


In [66]:
cria_graficos_barras_apro_repro(reprovados, 0, 'Reprovados disciplina 0')
cria_graficos_barras_apro_repro(reprovados, 1, 'Reprovados disciplina 1')
cria_graficos_barras_apro_repro(reprovados, 2, 'Reprovados disciplina 2')
cria_graficos_barras_apro_repro(reprovados, 3, 'Reprovados disciplina 3')
cria_graficos_barras_apro_repro(reprovados, 5, 'Reprovados disciplina 5')
cria_graficos_barras_apro_repro(reprovados, 6, 'Reprovados disciplina 6')

INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


INFO:bokeh.core.state:Session output file 'stacked_bar.html' already exists, will be overwritten.


## Preparação para determinar Correlação entre média de notas e distância até a UF

In [21]:
# Import pandas
import pandas as pd
import geocoder as gc
from tqdm import tqdm
from geopy.geocoders import Nominatim
from geopy.distance import vincenty

# Import BoxPlot, output_notebook, and show from bokeh.charts
from bokeh.charts import BoxPlot, Donut, Bar, Histogram, output_notebook, show
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend
from bokeh.layouts import gridplot, row
from bokeh.models import HoverTool
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import ColumnDataSource

In [22]:
#Verificar distancias
distancias = df[df["status"] == "ATIVO"].copy()
distancias["LAT"], distancias["LON"], distancias["KM"] = [0,0,0]
distancias = distancias.reset_index()
uf = gc.google("59064741").latlng

for i in tqdm(range(len(distancias))): ## trocar o range por range(len(distancias))
    st = distancias.loc[i,'CEP']
    g = gc.google(st)
    if g.lat == None:
        distancias.loc[i, "LAT"] = 0
    elif g.lng == None:
        distancias.loc[i, "LON"] = 0
    else:
        distancias.loc[i, "LON"] = g.lng
        distancias.loc[i, "LAT"] = g.lat
print("Completo")

distancias.to_csv('LatLong_Alunos3.csv', encoding="utf-8")
##Salva tabela criada
    
#UFRN 59064-741
distancias.head()

  0%|          | 5/3440 [00:03<43:16,  1.32it/s]


KeyboardInterrupt: 

In [38]:
distancias.reset_index()

,level_0,index,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,LAT,LON,KM
0,0,3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,-5.832998,-35.242542,3.451075
1,1,4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,-5.872282,-35.206600,4.027258
2,2,5,3,59088150,2014,1,ATIVO,2014,2,7.2,0,Aprovado,-5.872282,-35.206600,4.027258
3,3,8,5,59112430,2014,1,ATIVO,2014,2,8.9,0,Aprovado,-5.750546,-35.258952,10.830276
4,4,12,7,59020330,2014,1,ATIVO,2014,2,5.6,0,Aprovado,-5.798325,-35.202731,4.301033
5,5,13,8,59151610,2014,1,ATIVO,2014,2,1.5,0,Reprovado,0.000000,0.000000,0.000000
6,6,14,8,59151610,2014,1,ATIVO,2015,1,6.9,0,Aprovado,0.000000,0.000000,0.000000
7,7,21,12,59547000,2014,1,ATIVO,2014,2,2.6,0,Reprovado,-5.516642,-36.067595,101.201317
8,8,22,12,59547000,2014,1,ATIVO,2015,1,5.9,0,Aprovado,-5.516642,-36.067595,101.201317
9,9,25,14,59150500,2014,1,ATIVO,2014,2,1.2,0,Reprovado,0.000000,0.000000,0.000000


In [71]:
distancias =  pd.read_csv('LatLong_Alunos3.csv', encoding="utf-8", index_col=0)

for atual in tqdm(range(len(distancias))): ## trocar o range por range(len(distancias))
    lt = distancias.loc[atual, "LAT"]
    ln = distancias.loc[atual, "LON"]
    if lt != 0. and ln != 0.:
        compare = (lt, ln)
        #print(vincenty(uf, compare).km)
        distancias.loc[atual, "KM"] = vincenty(uf, compare).km
        
distancias.to_csv('LatLong_Alunos.csv', encoding="utf-8")    
distancias.head()

100%|██████████| 3440/3440 [00:04<00:00, 775.48it/s] 


,index,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,LAT,LON,KM
0,3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,-5.832998,-35.242542,3.451075
1,4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,-5.872282,-35.206600,4.027258
2,5,3,59088150,2014,1,ATIVO,2014,2,7.2,0,Aprovado,-5.872282,-35.206600,4.027258
3,8,5,59112430,2014,1,ATIVO,2014,2,8.9,0,Aprovado,-5.750546,-35.258952,10.830276
4,12,7,59020330,2014,1,ATIVO,2014,2,5.6,0,Aprovado,-5.798325,-35.202731,4.301033


## Determinar Correlação entre média de notas e distância até a UF 
#### Covariância -> (Σ[(xi-xmed)*(yi-ymed)])/n-1
#### Variância de cada variável (X,Y) -> Σ(xi-xmed)² e Σ(yi-ymed)² 
#### Correlação -> Cov(X,Y)/[√Va(X) * √Va(Y)]  


In [44]:
#Considera apenas as aprovações -> Validação 1
distancias = distancias[distancias["status.disciplina"] == 'Aprovado']

#Removendo distâncias desnecessárias -> Validação 2 e 3
distancias = distancias[distancias['KM'] != 0]
distancias = distancias[distancias['CEP'] != 0]

#Validação 4
distancias = distancias[distancias['KM'] < 30]

distancias

,index,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,LAT,LON,KM
0,3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,-5.832998,-35.242542,3.451075
2,5,3,59088150,2014,1,ATIVO,2014,2,7.2,0,Aprovado,-5.872282,-35.206600,4.027258
3,8,5,59112430,2014,1,ATIVO,2014,2,8.9,0,Aprovado,-5.750546,-35.258952,10.830276
4,12,7,59020330,2014,1,ATIVO,2014,2,5.6,0,Aprovado,-5.798325,-35.202731,4.301033
12,28,15,59151800,2014,1,ATIVO,2014,2,8.5,0,Aprovado,-5.890733,-35.196979,6.242204
14,36,20,59090620,2014,1,ATIVO,2014,2,6.4,0,Aprovado,-5.885479,-35.168988,7.204428
16,39,23,59070750,2014,2,ATIVO,2015,1,8.9,0,Aprovado,-5.833169,-35.231361,2.220069
17,41,25,59115685,2014,1,ATIVO,2016,1,7.3,0,Aprovado,-5.744671,-35.271933,12.132286
20,44,26,59290000,2014,1,ATIVO,2015,2,6.6,0,Aprovado,-5.776116,-35.368568,18.616898
21,45,27,59040240,2014,1,ATIVO,2014,1,5.9,0,Aprovado,-5.793517,-35.223136,4.891914


In [72]:
#Lista com ID dos alunos validados
alunos_validos = distancias.a_ID.unique()
alunos_validos

array([  2,   3,   5,   7,   8,  12,  14,  15,  16,  20,  21,  23,  25,
        26,  27,  30,  31,  33,  34,  37,  38,  39,  41,  42,  45,  47,
        48,  54,  55,  57,  60,  61,  62,  63,  64,  69,  70,  71,  72,
        73,  74,  77,  80,  83,  86,  87,  90,  91,  92,  93,  95,  98,
       101, 102, 103, 108, 110, 111, 112, 115, 116, 117, 120, 121, 123,
       125, 128, 129, 130, 131, 132, 133, 134, 136, 137, 139, 145, 146,
       147, 150, 152, 153, 154, 155, 156, 157, 158, 160, 162, 165, 166,
       188, 196, 197, 198, 201, 204, 208, 210, 218, 220, 228, 236, 237,
       238, 240, 242, 243, 245, 249, 253, 257, 259, 260, 261, 262, 263,
       264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 275, 276, 277,
       278, 279, 280, 281, 282, 284, 285, 286, 287, 289, 290, 291, 292,
       293, 294, 295, 296, 297, 300, 301, 302, 303, 304, 306, 307, 308,
       309, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 33

### Ocorrência dos dados

In [73]:
#Valores 'chave' da análise
valor_x_list = []
valor_y_list = []
for i in range(len(alunos_validos)):
    
    #Seleciona todas as ocorrências do aluno com aquele ID
    aluno = distancias[distancias["a_ID"] == alunos_validos[i]]    
    
    #Calcula média das notas para aquele aluno
    media =  aluno['nota'].mean()
    distancia_UF = aluno['KM'].mean()
    
    #Adiciona resultado à lista de distribuição X 
    if( media <= 6.0 ):
        valor_x_list.append(0)
    elif( media > 6.0 and media <= 7.0 ):
        valor_x_list.append(1)
    elif( media > 7.0 and media <= 8.0 ):
        valor_x_list.append(2)
    else:
        valor_x_list.append(3)
        
    #Adiciona resultado à lista de distribuição Y 
    if( distancia_UF <= 1.5 ):
        valor_y_list.append(0)
    elif( distancia_UF > 1.5 and distancia_UF <= 4.0 ):
        valor_y_list.append(1)
    elif( distancia_UF > 4.0 and distancia_UF <= 8.0 ):
        valor_y_list.append(2)
    else:
        valor_y_list.append(3)

## Tabela de Ocorrências

In [74]:
#Tabela de Ocorrências
distribuicao = pd.DataFrame(columns=('Valor X_Nota', 'Valor Y_Distancia', 'Xi - Xmed', 'Yi - Ymed'
                                     , 'Prod', '(Xi - Xmed)^2', '(Yi - Ymed)^2' ) )

distribuicao["Valor X_Nota"] = valor_x_list
distribuicao["Valor Y_Distancia"] = valor_y_list

distribuicao
#distancias["LAT"], distancias["LON"], distancias["KM"] = [0,0,0]

,Valor X_Nota,Valor Y_Distancia,Xi - Xmed,Yi - Ymed,Prod,(Xi - Xmed)^2,(Yi - Ymed)^2
0,0,1,NaN,NaN,NaN,NaN,NaN
1,0,2,NaN,NaN,NaN,NaN,NaN
2,2,3,NaN,NaN,NaN,NaN,NaN
3,0,2,NaN,NaN,NaN,NaN,NaN
4,1,0,NaN,NaN,NaN,NaN,NaN
5,0,3,NaN,NaN,NaN,NaN,NaN
6,0,0,NaN,NaN,NaN,NaN,NaN
7,3,2,NaN,NaN,NaN,NaN,NaN
8,2,3,NaN,NaN,NaN,NaN,NaN
9,0,2,NaN,NaN,NaN,NaN,NaN


### Cálculo do Xmed e Ymed (Valor médio de todos os pontos de X e Y)

In [75]:
xmed =  distribuicao['Valor X_Nota'].mean()
ymed =  distribuicao['Valor Y_Distancia'].mean()

### Cálculo (Xi - Xmed) e (Yi - Ymed) i = índice dos valores de X e Y

In [76]:
dif_X = []
dif_Y = []
prod = []

for i in range(len(alunos_validos)):
    
    #Calcula diferença para cada valor de X e salva numa lista
    difX = valor_x_list[i] - xmed
    
    dif_X.append(difX)
    
    #Calcula diferença para cada valor de Y e salva numa lista
    difY = valor_y_list[i] - ymed
    
    dif_Y.append(difY)
    
    #Calcula produto entre valores
    prod_Difs = difX*difY
    
    prod.append(prod_Difs)
    
#Adiciona na tabela
distribuicao["Xi - Xmed"] = dif_X
distribuicao["Yi - Ymed"] = dif_Y
distribuicao["Prod"] = prod

In [53]:
distribuicao

,Valor X_Nota,Valor Y_Distancia,Xi - Xmed,Yi - Ymed,Prod,(Xi - Xmed)^2,(Yi - Ymed)^2
0,1,1,-0.445283,-0.973585,0.433521,NaN,NaN
1,1,2,-0.445283,0.026415,-0.011762,NaN,NaN
2,2,3,0.554717,1.026415,0.569370,NaN,NaN
3,1,2,-0.445283,0.026415,-0.011762,NaN,NaN
4,3,2,1.554717,0.026415,0.041068,NaN,NaN
5,1,2,-0.445283,0.026415,-0.011762,NaN,NaN
6,2,1,0.554717,-0.973585,-0.540064,NaN,NaN
7,1,3,-0.445283,1.026415,-0.457045,NaN,NaN
8,0,3,-1.445283,1.026415,-1.483460,NaN,NaN
9,2,2,0.554717,0.026415,0.014653,NaN,NaN


### Calcula a covariância

In [77]:
passo1 = distribuicao['Prod'].sum()

covXY = passo1/(len(alunos_validos)-1)
covXY

0.1940652322049366

In [78]:
dif_X_quadrada = []
dif_Y_quadrada = []

for i in range(len(alunos_validos)):
    
    #Eleva cada diferença ao quadrado
    difX_quad = dif_X[i]*dif_X[i]
    
    dif_X_quadrada.append(difX_quad)
    
    #Eleva cada diferença ao quadrado
    difY_quad = dif_Y[i]*dif_Y[i]
    
    dif_Y_quadrada.append(difY_quad)
    
#Adiciona na tabela
distribuicao["(Xi - Xmed)^2"] = dif_X_quadrada
distribuicao["(Yi - Ymed)^2"] = dif_Y_quadrada

In [79]:
distribuicao

,Valor X_Nota,Valor Y_Distancia,Xi - Xmed,Yi - Ymed,Prod,(Xi - Xmed)^2,(Yi - Ymed)^2
0,0,1,-0.573854,-0.42275,0.242597,0.329308,0.178718
1,0,2,-0.573854,0.57725,-0.331257,0.329308,0.333217
2,2,3,1.426146,1.57725,2.249388,2.033892,2.487716
3,0,2,-0.573854,0.57725,-0.331257,0.329308,0.333217
4,1,0,0.426146,-1.42275,-0.606299,0.181600,2.024219
5,0,3,-0.573854,1.57725,-0.905111,0.329308,2.487716
6,0,0,-0.573854,-1.42275,0.816451,0.329308,2.024219
7,3,2,2.426146,0.57725,1.400492,5.886184,0.333217
8,2,3,1.426146,1.57725,2.249388,2.033892,2.487716
9,0,2,-0.573854,0.57725,-0.331257,0.329308,0.333217


### Calcula Variância de X e Y

In [80]:
passo21 = distribuicao['(Xi - Xmed)^2'].sum()
passo22 = distribuicao['(Yi - Ymed)^2'].sum()

#Variância de X
varX = passo21/(len(alunos_validos)-1)

#Variância de Y
varY = passo22/(len(alunos_validos)-1)


In [81]:
varX

0.8708118221821896

In [82]:
varY

1.7138352794428442

## Executar passo 3

### Calcular Correlação

In [83]:
from math import sqrt

desvio_padraoX = sqrt(varX)
desvio_padraoY = sqrt(varY)

#Calculo da correlação
corrXY = covXY/(desvio_padraoX*desvio_padraoY)

corrXY

0.15885505481053944